In [12]:
import pandas as pd
import numpy as np
import bqplot
import yt

Symbol_T = 'Time'
Symbol_S = 'Total Sightings'
Symbol_D = 'Total Duration'

In [5]:
from bqplot import DateScale, LinearScale, Axis, Lines, Scatter, Bars, Hist, Figure
from bqplot.interacts import (
    FastIntervalSelector, IndexSelector, BrushIntervalSelector,
    BrushSelector, MultiSelector, LassoSelector, PanZoom, HandDraw
)
from traitlets import link

from ipywidgets import ToggleButtons, VBox, HTML, Dropdown

In [6]:
names = ["date", "city", "state", "country", "shape", "duration_seconds",
         "duration_reported", "description", "report_date", "latitude",
         "longitude"]
fn = "/srv/nbgrader/data/ufo-scrubbed-geocoded-time-standardized.csv"
ufo = pd.read_csv(fn, names = names, parse_dates = ["date", "report_date"])

In [8]:
total_sightings = ufo.groupby(['state','date'])['report_date'].count()
total_time = ufo.groupby(['state','date'])['duration_seconds'].sum()

### Define a Figure

In [26]:
total_S = total_sightings['tx'].to_dict()
total_D = total_time['tx'].to_dict()

In [31]:
total_S.values()

dict_values([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [51]:
dt_x_fast = DateScale()
lin_y = LinearScale()

x_ax = Axis(label = 'Year', scale = dt_x_fast)
y_ay_S = Axis(label = (Symbol_S + ' of the selected state'),
           scale = lin_y, orientation = 'vertical')
y_ay_D = Axis(label = (Symbol_D + ' of the selected state'),
           scale = lin_y, orientation = 'vertical')
lc = Lines(x = list(total_S.keys()), y = list(total_S.values()), 
           scales = {'x' : dt_x_fast, 'y' : lin_y},
          colors = ['orange'])
lc2 = Lines(x = list(total_D.keys()), y = list(total_D.values()),
           scales = {'x' : dt_x_fast, 'y' : lin_y},
           colors = ['blue'])

### Define the type of selector we would like

In [45]:
intsel_fast = FastIntervalSelector(scale = dt_x_fast, marks = [lc,lc2])


### Define a function that will be called when the FastIntervalSelector is interacted with

In [42]:
md = {}
def fast_interval_change_callback(change):
    ind = pd.to_datetime(change.new)
    tot = ufo.loc[ind[0]:ind[1]]["Time"].sum()
    db_fast.value = 'The selected time period is '+ str(change.new) + 'total:' + str(tot)

### Connect the selectors to the function


In [46]:
intsel_fast.observe(fast_interval_change_callback,names = ['selected'])

### Use the HTML widget to see the value of what we are selecting and modify it when an interaction is performed on the selector

In [52]:
db_fast = HTML()
db_fast.value = 'The selected time period is ' + str(intsel_fast.selected)


fig_fast_insel_sights = Figure(marks = [lc], axes = [x_ax, y_ay_S],
                               interaction = intsel_fast)
fig_fast_insel_time = Figure(marks = [lc2], axes = [x_ax, y_ay_D],
                            interaction = intsel_fast)

def watch_dropdown(change):
    db_fast.value = str((change.old, change.new))
new_dropdown = Dropdown(options = ['Total Sightings','Total Time'])
new_dropdown.observe(watch_dropdown, names = ['value'])
VBox([db_fast,fig_fast_insel_sights, new_dropdown])


VBox(children=(HTML(value='The selected time period is None'), Figure(axes=[Axis(label='Year', scale=DateScale(), side='bottom'), Axis(label='Total Sightings of the selected state', orientation='vertical', scale=LinearScale(), side='left')], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, interaction=FastIntervalSelector(marks=[Lines(colors=['orange'], interactions={'hover': 'tooltip'}, scales={'x': DateScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([Timestamp('1910-06-01 15:00:00'), Timestamp('1945-07-15 14:00:00'),
       Timestamp('1946-03-15 15:30:00'), ...,
       Timestamp('2014-05-03 20:54:00'), Timestamp('2014-05-03 23:11:00'),
       Timestamp('2014-05-04 00:00:00')], dtype=object), y=array([1, 1, 1, ..., 1, 1, 1])), Lines(colors=['blue'], interactions={'hover': 'tooltip'}, scales={'x': DateScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([Timestamp('1910-06-01 15:00:00'), Timestamp('1945-07-15 14:00:00'),
       Timestamp('1946-03-15 15:30:00'), ...,
       Timestamp('2014-05-03 20:54:00'), Timestamp('2014-05-03 23:11:00'),
       Timestamp('2014-05-04 00:00:00')], dtype=object), y=array([  1.20000000e+02,   7.20000000e+03,   2.00000000e+01, ...,
         1.80000000e+03,   6.00000000e+00,   2.00000000e+00]))], scale=DateScale()), layout=Layout(min_width='125px'), marks=[Lines(colors=['orange'], interactions={'hover': 'tooltip'}, scales={'x': DateScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([Timestamp('1910-06-01 15:00:00'), Timestamp('1945-07-15 14:00:00'),
       Timestamp('1946-03-15 15:30:00'), ...,
       Timestamp('2014-05-03 20:54:00'), Timestamp('2014-05-03 23:11:00'),
       Timestamp('2014-05-04 00:00:00')], dtype=object), y=array([1, 1, 1, ..., 1, 1, 1]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0)), Dropdown(options=('Total Sightings', 'Total Time'), value='Total Sightings')))